### GPT - Generatively Pretrained Transformer

In [41]:
import numpy as np
import pandas as pd
import wget
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import os
os.path.dirname(__file__)

NameError: name '__file__' is not defined

In [4]:
# !wget('https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt')

In [3]:
with open('../data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print('total number of characters in the dataset: ', len(text))

total number of characters in the dataset:  1115394


In [5]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
vocab = sorted(list(set(text)))
vocab_size = len(vocab)
print(''.join(vocab))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
stoi = {s:i for i, s in enumerate(vocab)}
itos = {i:s for s, i in stoi.items()}
encode = lambda seq: [stoi[s] for s in seq]
decode = lambda seq: ''.join([itos[i] for i in seq])

In [8]:
print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [25]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

n = int(data.shape[0] * 0.9)
train_data = data[:n]
val_data = data[n:]
print(train_data.shape, val_data.shape)

torch.Size([1115394]) torch.int64
torch.Size([1003854]) torch.Size([111540])


In [26]:
block_size = 8    # context length
sample = train_data[:block_size+1]
print(sample)

for t in range(block_size):
    context = sample[0:t+1]
    target = sample[t+1]
    print(f'when context is {context.tolist()}, then target is {target}')

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
when context is [18], then target is 47
when context is [18, 47], then target is 56
when context is [18, 47, 56], then target is 57
when context is [18, 47, 56, 57], then target is 58
when context is [18, 47, 56, 57, 58], then target is 1
when context is [18, 47, 56, 57, 58, 1], then target is 15
when context is [18, 47, 56, 57, 58, 1, 15], then target is 47
when context is [18, 47, 56, 57, 58, 1, 15, 47], then target is 58


In [33]:
torch.manual_seed(1337)
batch_size = 5
block_size = 8

In [30]:
def get_batch(split, batch_size=5):
    data = train_data if split == 'train' else val_data
    start_idxs = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in start_idxs])
    y = torch.stack([data[i+1:i+block_size+1] for i in start_idxs])
    return x, y

x_batch, y_batch = get_batch('train')
print(x_batch.shape, y_batch.shape)
print(x_batch)
print(y_batch)

for x, y in zip(x_batch, y_batch):
    for t in range(block_size):
        context = x[0:t+1]
        target = y[t]
        print(f'when context is {context}, target is {target}')

torch.Size([5, 8]) torch.Size([5, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54],
        [57, 43, 60, 43, 52,  1, 63, 43]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39],
        [43, 60, 43, 52,  1, 63, 43, 39]])
when context is tensor([24]), target is 43
when context is tensor([24, 43]), target is 58
when context is tensor([24, 43, 58]), target is 5
when context is tensor([24, 43, 58,  5]), target is 57
when context is tensor([24, 43, 58,  5, 57]), target is 1
when context is tensor([24, 43, 58,  5, 57,  1]), target is 46
when context is tensor([24, 43, 58,  5, 57,  1, 46]), target is 43
when context is tensor([24, 43, 58,  5, 57,  1, 46, 43]), target is 39
when context is tensor([44]), target is 53
when context is tensor([44, 53]), target is

In [31]:
x_batch.max(dim=1)[0]

tensor([58, 58, 58, 54, 63])

In [87]:
out - out.max(dim=2)[0].unsqueeze(-1)

tensor([[[-4.1437, -1.5925, -2.6091,  ..., -1.7312, -1.3890, -2.8179],
         [-2.2564, -2.0667, -4.2918,  ..., -2.2255, -2.8801, -2.9464],
         [-3.1895, -3.1137, -1.0885,  ..., -4.1962, -2.3154, -2.0492],
         ...,
         [-0.4255, -1.6778, -2.0062,  ..., -1.8107, -2.0980, -2.6525],
         [-0.1581, -1.7713, -0.5359,  ..., -3.2353, -1.7776, -1.6778],
         [-2.2564, -2.0667, -4.2918,  ..., -2.2255, -2.8801, -2.9464]],

        [[-2.0594, -2.6820, -2.6960,  ..., -3.4306, -5.4442, -3.8636],
         [-2.2182, -1.6790, -3.2129,  ..., -1.8255, -2.6331, -2.5374],
         [-2.1677, -4.7835, -1.5791,  ...,  0.0000, -1.5278, -0.6059],
         ...,
         [-4.3497, -5.1640, -4.3130,  ..., -2.8290, -4.5995, -3.6540],
         [-2.5800, -2.1846, -3.3516,  ..., -0.1258, -2.6425, -2.8347],
         [-2.1156, -2.6152, -2.6351,  ..., -2.8638, -1.8345, -3.4606]],

        [[-2.5800, -2.1846, -3.3516,  ..., -0.1258, -2.6425, -2.8347],
         [-0.1581, -1.7713, -0.5359,  ..., -3

### Bigram Language Model

In [37]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)    # [B,T,C]

        if targets is None:
            loss = None
        else:
            # counts = (logits - logits.max(dim=2)[0].unsqueeze(-1)).exp()
            # probs = counts / counts.sum(dim=2, keepdims=True)  # [B,T,C]
            # loss2 = -probs[:, :, targets].log().mean()

            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx)  # [B,T,C]
            logits = logits[:, -1, :]    # [B,C]
            probs = F.softmax(logits, dim=-1)    # [B,C]
            idx_next = torch.multinomial(probs, num_samples=1)    # [B,1]
            idx = torch.cat([idx, idx_next], dim=1)
            # print(idx.shape)
        return idx


model = BigramLanguageModel(vocab_size)
out, loss = model(x_batch, y_batch)
print(out.shape)
print(loss)

torch.Size([40, 65])
tensor(4.5264, grad_fn=<NllLossBackward0>)


In [38]:
inp = torch.zeros((1,1), dtype=torch.long)
print(decode(model.generate(idx=inp, max_new_tokens=1000)[0].tolist()))



bb3r,Z3vrBwgJQW XeT &?aoV'.WgZWh-H,A'wh?q!eF&:dh:dTFPuWcJloiAbUvwoIuvCEgvl3AOGkKmb&WDsprzJFPFqQWSe-YP?zQNrzXr-O3NNG&bgWhq?oZDmB':hWQq-'ECt$!S.!u;C$gyYHQDwiBuk
DsEb$KofVFtTbIxQYRwkwiHznhoqsI?iPH:PFNUerDh?SyfAbB-w3AsVwDDh.mHWFUsoJqB$?MEQNZ$Vuis-mBT3pxBlEoAbNBCCQKjr$Nm:b
?ndag;Dsk,-xk$?zay,oEO,DuFPFoV3H:SAgCDtcq
-ux
P'gWI,MasQlPM'gK$gyCxwF?Nii.mr;V,R E;e&,dFgxqDtWTEJOSKnztae'WwTFeTmhkmc,bBxUJ!GbKVM.gZCN3&f-PLiB-U&daEJC lHYzUdfyfd.MaAbtkp.lCm?P$cttUwDALJRkARA'cnKoZv.rUDc.!c.oagW,R,lLtNBw-ux!,yZnYeARD$nXIWCu,ou,dD.o,-RnnZLEHgcM
MebyYByf&WLRG 'H:mIv?o!pWHDHe3ARjYa,Yzt$dxLHqI-liXc3Xy-t$dTgt$sj,gg'Xylof ChfX'yYeFgJ:XAoA.EZ$gWEknU$CbvSEdI  ozY?akLu,oKgm&-xOuBIOPYTjQCk:MbZvlou 'Uv3'M
xQDhtHvBQQx?zH$-u-t&LB'yJ!&WSKDHIx!tprwC Ou-u 'Hq;P!
cxOPmbyc!mjzRo'YE?zsEBI,uPQDnZAF IRs 
X;nYM,gH:PLpMXEZ$:
PzG &QAefPcldcSJCHDJK'ySh?fZ$V!pV!pB-loQQKg
,zvXJeePKGFb;Af MQAMGGGTSevXAoYTfDqShkdz,OSb3-TBu,sfOkcWbOQAtUsAzcLsPwielAw;dzJVGmcTtMIV!ag&$rB'.i-xrUGMHYTOHoPD
kQqjdySSJVHtLrHDjQhHOiG.;Whp&DwFU$NRYT!mlJeupeV3

In [39]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [42]:
batch_size = 32
for itr in tqdm(range(50000)):
    x_batch, y_batch = get_batch('train', batch_size)

    out, loss = model(x_batch, y_batch)
    # for param in model.parameters():
    #     param.grad = None
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

100%|██████████| 50000/50000 [00:45<00:00, 1102.04it/s]

2.425154447555542


In [43]:
inp = torch.zeros((1,1), dtype=torch.long)
print(decode(model.generate(idx=inp, max_new_tokens=1000)[0].tolist()))


S:
BRWishs lay
KIOrlshanlindate irand pl m d? sccrveofod le
METht ich the st arreral ivernonyom fe t if teairt pimom. ot buingo n fouly cqud s thio, acrg; s.
Tr fowa sshekn Paned: sont d k?
Br torero,-cee,
NRARIUCo thastwhe t on cend: iooshugf shen t beand;
Oniomye t deive.
Fougin.

My ton, ES:
ANONonflaf arnodesthoo hoo!
Al by
Tot perne the emad fore E toushayak s od lld hathy te t halousht, y soo w, n'd tho by on henss, canama e se rone; is tothothaus pt s aro'
I ty'th.
HANowid tis
Dolir maintha t
S:
YO: keazeaken oun: nt mun.
Winoun?
Ofeniowesthounse hilort ckirighirdovensomar
Towndithid puitariowo Ithind:
Ha blit e he 'd d
mep,
Aloilo,

G gr ABUng, indlin.
PUCENGLI he KIf thenave waryokiey tl noun w?


Whivenat herefe man.
T:
Bin her.
ARGLLINags, eviva te cos omoronoprmpin hend br h br re.



D pisor hounde thablllfragen he fuy, a l. lopry thothenty blaf b the
CE:
coty nes.
QUFIZARGle s?
AUCHil ang habucokssovommor.

Ifo g d belo ngol t:

RI:
La s blipin me.
Theer moroneancorst d 

In [57]:
# toy example
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 32])

In [79]:
# version 1
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]    # [t+1,C]
        xbow[b, t] = torch.mean(xprev, dim=0)
# xbow

In [80]:
a = torch.tril(torch.ones((3,3)))
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b

print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[4., 5.],
        [2., 7.],
        [1., 0.]])
tensor([[4.0000, 5.0000],
        [3.0000, 6.0000],
        [2.3333, 4.0000]])


In [81]:
# version 2
wei = torch.tril(torch.ones((T, T)))
wei = wei / torch.sum(wei, dim=1, keepdims=True)
print(wei)
xbow2 = wei @ x     # (B, T, T) @ (B, T, C) --> (B, T, C)
xbow2.shape

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


torch.Size([4, 8, 32])

In [83]:
print(torch.allclose(xbow, xbow2))
print((xbow - xbow2).sum())

True
tensor(-5.1456e-07)


In [85]:
# version 3
tril = torch.tril(torch.ones((T, T)))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
# print(wei)
xbow3 = wei @ x
print(torch.allclose(xbow, xbow3))
xbow3.shape

True


torch.Size([4, 8, 32])

In [69]:
tmp = nn.Embedding(65, 32)
tmp1 = torch.arange(32)
tmp(tmp1).shape

torch.Size([32, 32])

In [70]:
n_embd = 32
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.pos_embedding_table = nn.Embedding(block_size, n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        tok_emb = self.token_embedding_table(idx)    # (B,T,C=n_embd)
        pos_emb = self.pos_embedding_table(torch.arange(block_size))    # (T,C=n_embd)
        x = tok_emb + pos_emb    # (B,T,C)
        logits = self.lm_head(x)    # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            # counts = (logits - logits.max(dim=2)[0].unsqueeze(-1)).exp()
            # probs = counts / counts.sum(dim=2, keepdims=True)  # [B,T,C]
            # loss2 = -probs[:, :, targets].log().mean()

            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx)  # [B,T,C]
            logits = logits[:, -1, :]    # [B,C]
            probs = F.softmax(logits, dim=-1)    # [B,C]
            idx_next = torch.multinomial(probs, num_samples=1)    # [B,1]
            idx = torch.cat([idx, idx_next], dim=1)
            # print(idx.shape)
        return idx


model = BigramLanguageModel(vocab_size)
out, loss = model(x_batch, y_batch)
print(out.shape)
print(loss)

torch.Size([256, 65])
tensor(4.5233, grad_fn=<NllLossBackward0>)


each token will emit two vectors
query vector for each token - what I am looking for
key vector for each token - what do I contain

In [102]:
# version 4: self-attention (is a communication mechanism)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

# single-head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
val = nn.Linear(C, head_size, bias=False)
k = key(x)    # (B,T,16)
q = query(x)    # (B,T,16)
wei = q @ k.transpose(-2,-1)    # (B,T,16) @ (B,16,T) --> (B,T,T)

tril = torch.tril(torch.ones(T,T))  # (T,T)
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))  # (B,T,T)
wei = F.softmax(wei, dim=-1)  # (B,T,T)

v = val(x)    # (B,T,16)
out = wei @ v  # (B,T,T) @ (B,T,16) --> (B,T,16)
out.shape

torch.Size([4, 8, 16])

In [119]:
[i * 2 for i in range(10)]

SyntaxError: can't use starred expression here (1713185751.py, line 1)

In [103]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4365, 0.5635, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0202, 0.2026, 0.7771, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5387, 0.1885, 0.0444, 0.2285, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2233, 0.0607, 0.0150, 0.5439, 0.1571, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.1686, 0.0208, 0.0668, 0.4425, 0.1012, 0.0000, 0.0000],
        [0.1184, 0.1106, 0.0032, 0.5313, 0.0572, 0.0592, 0.1199, 0.0000],
        [0.1300, 0.0152, 0.0033, 0.0166, 0.1057, 0.0346, 0.4233, 0.2713]],
       grad_fn=<SelectBackward0>)

In [114]:
k = torch.randn(B,T,head_size)  # (B,T,head_size)
q = torch.randn(B,T,head_size)  # (B,T,head_size)

wei = q @ k.transpose(-2,-1) / (head_size**0.5)  # (B,T,T)
wei.shape

torch.Size([4, 8, 8])

In [115]:
k.var()

tensor(0.9252)

In [116]:
q.var()

tensor(1.1416)

In [117]:
wei.var()

tensor(1.0104)